# Lectutorial 3: Design choices and hyperparameter tuning

1. Activation functions
2. Weights initialisation
3. Regularisation
4. Hyper-parameter tuning with Optuna
5. Learning rate schedules
6. Batch-norm
7. Faster optimisers

In [ ]:
from copy import deepcopy
import numpy as np
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import optuna
from functools import partial

## Coding Time 1: Activation functions, weights initialisation & regularisation

### 1. Activation functions

Let's train a small multi-layer neural network (also called multi-layer perceptron or MLP) on the california housing dataset and try out different activation functions.

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

#### Data loading and pre-processing

In [ ]:
housing_dataset = fetch_california_housing()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(housing_dataset.data, housing_dataset.target, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)

X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)
X_test = torch.FloatTensor(X_test)

y_train = torch.FloatTensor(y_train)
y_valid = torch.FloatTensor(y_valid)
y_test = torch.FloatTensor(y_test)

In [ ]:
means = X_train.mean(axis=0, keepdims=True)
stds = X_train.std(axis=0, keepdims=True)
X_train = (X_train - means) / stds
X_valid = (X_valid - means) / stds
X_test = (X_test - means) / stds

y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

#### Defining and training a model

First we create a class to instantiate model with a simple architecture using 2 hidden layer with ReLU activation functions. As we are doing regression, we can leave the output layer as a linear activation (already applied in the `nn.Linear` layer). 

In [ ]:
class MLP_ReLU(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_hidden3, n_outputs):
      super().__init__()
      self.mlp = nn.Sequential(
          nn.Flatten(),
          nn.Linear(n_inputs, n_hidden1),
          nn.ReLU(),
          nn.Linear(n_hidden1, n_hidden2),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_hidden3),
          nn.ReLU(),
          nn.Linear(n_hidden3, n_outputs)
          )
    def forward(self, X):
        return self.mlp(X)


Training loop returning training and validation loss:

In [ ]:
def train(model, optimizer, loss_fn, train_loader, valid_loader, n_epochs):
    train_losses = []
    valid_losses = []

    for epoch in range(n_epochs):
        #Training
        model.train()
        epoch_train_loss = 0.
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            y_train_pred = model(X_train_batch)
            train_loss = loss_fn(y_train_pred, y_train_batch)
            epoch_train_loss += train_loss.item()
            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        mean_epoch_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(mean_epoch_train_loss)

        # Validation
        model.eval()
        epoch_valid_loss = 0.
        with torch.no_grad():
            for X_valid_batch, y_valid_batch in valid_loader:
                X_valid_batch, y_valid_batch = X_valid_batch.to(device), y_valid_batch.to(device)
                y_valid_pred = model(X_valid_batch)
                valid_loss = loss_fn(y_valid_pred, y_valid_batch)
                epoch_valid_loss += valid_loss.item()
        mean_epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        valid_losses.append(mean_epoch_valid_loss)

        print(f"Epoch {epoch + 1}/{n_epochs}, Training Loss: {mean_epoch_train_loss:.4f}, Valid Loss: {mean_epoch_valid_loss:.4f}")

    return (train_losses, valid_losses)

Conversion of the training and validation sets to `TensorDataset` and creation of the dataloader to use mini-batch gradient descent:

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataset = TensorDataset(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

Instanciation of a model from the previously declared class `MLP_ReLU` and selection of the learning rate, optimiser, loss (MSE) and number of epochs:

In [ ]:
model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse = nn.MSELoss()
n_epochs = 30

Model training and storage of the trainig and validation losses:

In [ ]:
train_losses_relu, valid_losses_relu = train(model, optimizer, mse, train_loader, valid_loader, n_epochs)

Plot the train and validation losses:

In [ ]:
plt.plot(train_losses_relu, label = "ReLU Train Loss")
plt.plot(valid_losses_relu, label = "ReLU Valid Loss")
plt.legend()
plt.grid()
plt.show()

**TODO**: Let's now create a new class `MLP_Sigmoid` to create similar models but with Sigmoid activation instead of ReLU. Complete the following class definition by adding the Sigmoid activation function for the 2 hidden layers. 

In [ ]:
class MLP_Sigmoid(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_outputs):
      super().__init__()
      self.mlp = nn.Sequential(
            nn.Linear(n_inputs, n_hidden1),
            # TODO
            nn.Linear(n_hidden1, n_hidden2),
            # TODO
            nn.Linear(n_hidden2, n_outputs)
            # TODO
            nn.Linear(n_hidden2, n_hidden3),
            )
    def forward(self, X):
        return self.mlp(X)

**TODO**: 
- Train a model instantiated from the `MLP_Sigmoid` class, using the same other settings (learning rate; optimiser, loss and number of epochs) than previously.
- Plot the training and validation losses for the model using ReLU and the model using Sigmoid on the same graph.

In [ ]:
# TODO

**TODO**: Have a look at the list of activation functions available in PyTorch and try out another one (e.g., Leaky ReLU). 

#### Output activation functions

The output activation function depends on the problem you are trying to solve:
- Regression: Linear activation with MSE, RMSE or MAE loss.
- Binary/Multilabel classification: Sigmoid with Binary Cross Entropy Loss.
- Multi-class classification: Softmax with Cross Entropy Loss.

**Important**: 
When doing binary/multilabel classification, either:
- Apply a `nn.Sigmoid()` activation function to your output layer and use `nn.BCELoss` as loss function, OR
- Do not apply `nn.Sigmoid()` activation function to your output layer and use `nn.BCEWithLogitsLoss` as loss function (applies sigmoid before BCE loss).
  In that case, you also will need to apply sigmoid to the output of the model at inference timle (after training).

When doing multi-class classification: 
- Do not apply `nn.Softmax()` activation function to your output as it is already applied in `nn.CrossEntropyLoss`.
- Apply the softmax function at inference time (after training). 


In [ ]:
# MULTILABEL CLASSIFICATION 1 (also works for Binary classification)
# Either Sigmoid + BCELoss
class MultiLabelClassifier(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_hidden3, n_classes):
      super().__init__()
      self.mlp = nn.Sequential(
          nn.Flatten(),
          nn.Linear(n_inputs, n_hidden1),
          nn.ReLU(),
          nn.Linear(n_hidden1, n_hidden2),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_hidden3),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_classes)
          nn.Sigmoid()
      )
    def forward(self, X):
        return self.mlp(X)

model = MultiLabelClassifier(n_inputs=10, n_hidden1=50, n_hidden2=40, n_classes=5)
criterion = nn.BCELoss()  # Do not Automatically applies sigmoid
# ...

In [ ]:
# MULTILABEL CLASSIFICATION 2 (also works for Binary classification)
# Or No Sigmoid + BCEWithLogitsLoss

class MultiLabelClassifier(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_hidden3, n_classes):
      super().__init__()
      self.mlp = nn.Sequential(
          nn.Flatten(),
          nn.Linear(n_inputs, n_hidden1),
          nn.ReLU(),
          nn.Linear(n_hidden1, n_hidden2),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_hidden3),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_classes)
          # No nn.Sigmoid here
      )
    def forward(self, X):
        return self.mlp(X)

model = MultiLabelClassifier(n_inputs=10, n_hidden1=50, n_hidden2=40, n_classes=5)
criterion = nn.BCEWithLogitsLoss()  # Automatically applies sigmoid
# ...

# At inference time
new_input = ... # Some new input
prediction = torch.sigmoid(model(new_input))

In [ ]:
# MULTI-CLASS CLASSIFICATION 

class MultiClassClassifier(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_hidden3, n_classes):
      super().__init__()
      self.mlp = nn.Sequential(
          nn.Flatten(),
          nn.Linear(n_inputs, n_hidden1),
          nn.ReLU(),
          nn.Linear(n_hidden1, n_hidden2),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_hidden3),
          nn.ReLU(),
          nn.Linear(n_hidden2, n_classes)
          # No nn.Softmax here
      )
    def forward(self, X):
        return self.mlp(X)

model = MultiLabelClassifier(n_inputs=10, n_hidden1=50, n_hidden2=40, n_classes=5)
criterion = nn.CrossEntropyLoss()  # Automatically applies softmax
# ...

# At inference time
new_input = ... # Some new input
prediction = torch.softmax(model(new_input))

## 2. Weights initialisation

The way the weights are initialised plays an important role to allow the signal to "flow" properly during the forward pass and the backpropagation of the gradients. I.e., we want to avoid values of activations and gradients to become too large or too small. This can be achieved by controling the variance or the bonds of the distribution from which the weights' values are drawn from. This helps to maintain a similar variance of ativations across the different layers and mitigate vanishing or exploding gradients.

There are two main initialisations used for the weights: 
- Xavier/Glorot initialisation for linear, tanh, sigmoid and softmax activation functions.
- Kaiming/He initialisation for ReLU and its variants (mitigate Dying ReLU problem).

Weights's values can be drawn from a normal or uniform distribution with variance/bounds calculated based on the number of neurons in the previous and next layer for Xavier/Glorot, or only based on the previous layer for Kaiming/He. 

Usually, drawing weigths' values from a uniform distribution is prefered for smaller networks, while drawing them from a normal distribution is prefered from deeper networks. 

The `nn.init` module contains several types of initialisation methods, including Xavier/Glorot and Kaiming/He uniform and normal. 

The following functions takes a module (i.e., a model) as input and apply Kaiming/He uniform or normal initialisation to the weigths. The biases are initialised to 0. It is acceptatble as the biases are not contributing to the symmetry problem as weigths do. Their primary role is to provide an adjustable threshold for neuron activation, so they can start from 0. 

In [ ]:
def use_he_uniform_init(module):
    if isinstance(module, nn.Linear):
        nn.init.kaiming_uniform_(module.weight)
        nn.init.zeros_(module.bias)

In [ ]:
def use_he_normal_init(module):
    if isinstance(module, nn.Linear):
        nn.init.kaiming_normal_(module.weight)
        nn.init.zeros_(module.bias)

**TODO**: Create similar functions for Xavier/Glorot initialisation.

In [ ]:
# TODO

You can then use them after instantiating a model:

In [ ]:
model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
model.apply(use_he_uniform_init)

## 3. Regularisation 

#### 3.1. Dropout

Let's add some dropout in the input and hidden layers. 

In [ ]:
class MLP_Dropout(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_hidden3, n_outputs):
      super().__init__()
      self.mlp = nn.Sequential(
            nn.Dropout(p=0.2), nn.Linear(n_inputs, n_hidden1),
            nn.ReLU(),
            nn.Dropout(p=0.2), nn.Linear(n_hidden1, n_hidden2),
            nn.ReLU(),
            nn.Dropout(p=0.2), nn.Linear(n_hidden2, n_hidden3),
            nn.ReLU(),
            nn.Dropout(p=0.2), nn.Linear(n_hidden3, n_outputs)
            )
    def forward(self, X):
        return self.mlp(X)

*Remark*: As dropout is behaving differently between training and evaluation, it is crutial to include the `model.train()` and `model.eval()` instructions in your training loop, to switch the model in training and evalaution mode, as we have already done.

In [ ]:
model = MLP_Dropout(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3=50, n_outputs=1).to(device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse = nn.MSELoss()
n_epochs = 30

In [ ]:
train_losses_drop, valid_losses_drop = train(model, optimizer, mse, train_loader, valid_loader, n_epochs)

In [ ]:
plt.plot(train_losses_relu, label = "ReLU no dropout Train Loss")
plt.plot(valid_losses_relu, label = "ReLU no dropout Valid Loss")
plt.plot(train_losses_drop, label = "ReLU dropout Train Loss")
plt.plot(valid_losses_drop, label = "ReLU dropout Valid Loss")
plt.legend()
plt.grid()
plt.show()

**TODO**: Looking at the train and validation losses with dropout activated can be misleading. What do you observe? Does this makes sense given we are using dropout?

The effect of dropout as regularisation is not so visible as our network and task are not too complex, with not much overfitting. However, using dropout can lead to higher validation performance (i.e., lower validation loss) with deeper networks and more complex problems prone to overfitting. 

#### 3.2. L1 and L2 regularisation

To implement L1 and L2 regularisation, you need to modify the loss in the training function.

In [ ]:
def train_reg(model, optimizer, loss_fn, train_loader, valid_loader, n_epochs, reg_type = "l1", alpha_reg = 1e-4):
    train_losses = []
    valid_losses = []
    params_to_regularize = [param for name, param in model.named_parameters() if not "bias" in name and not "bn" in name]
    
    for epoch in range(n_epochs):
        #Training
        model.train()
        epoch_train_loss = 0.
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            y_train_pred = model(X_train_batch)

            if reg_type == "l1":
                # l1 Regularisation
                main_loss = loss_fn(y_train_pred, y_train_batch)
                l1_loss = sum(param.abs().sum() for param in params_to_regularize)
                train_loss = main_loss + alpha_reg * l1_loss
            else:
                # l2 Regularisation
                main_loss = loss_fn(y_train_pred, y_train_batch)
                l2_loss = sum(param.pow(2.0).sum() for param in params_to_regularize)
                train_loss = main_loss + alpha_reg * l2_loss
            
            epoch_train_loss += train_loss.item()
            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        mean_epoch_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(mean_epoch_train_loss)

        # Validation
        model.eval()
        epoch_valid_loss = 0.
        with torch.no_grad():
            for X_valid_batch, y_valid_batch in valid_loader:
                X_valid_batch, y_valid_batch = X_valid_batch.to(device), y_valid_batch.to(device)
                y_valid_pred = model(X_valid_batch)
                valid_loss = loss_fn(y_valid_pred, y_valid_batch)
                epoch_valid_loss += valid_loss.item()
        mean_epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        valid_losses.append(mean_epoch_valid_loss)

        print(f"Epoch {epoch + 1}/{n_epochs}, Training Loss: {mean_epoch_train_loss:.4f}, Valid Loss: {mean_epoch_valid_loss:.4f}")

    return (train_losses, valid_losses)

In [ ]:
model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse = nn.MSELoss()
n_epochs = 30

In [ ]:
train_losses_l2, valid_losses_l2 = train_reg(model, optimizer, mse, train_loader, valid_loader, n_epochs, "l2", 1e-4)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_losses_relu, label = "ReLU no L2 reg Train Loss")
plt.plot(valid_losses_relu, label = "ReLU no L2 reg Valid Loss")
plt.legend()
plt.grid()
plt.subplot(2, 1, 2)
plt.plot(train_losses_l2, label = "ReLU L2 reg Train Loss")
plt.plot(valid_losses_l2, label = "ReLU L2 reg Valid Loss")
plt.legend()
plt.grid()
plt.show()

#### 3.3. Early stopping

Below is a simple implementation of early stopping i nthe training loop. It simply stops the training if the validation loss for an epoch is higher than in the previous epoch, and reverts the model to the model from the previous epoch. 

In [ ]:
def train_with_early_stopping(model, optimizer, loss_fn, train_loader, valid_loader, n_epochs):
    train_losses = []
    valid_losses = []

    for epoch in range(n_epochs):
        #Training
        model.train()
        epoch_train_loss = 0.
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            y_train_pred = model(X_train_batch)
            train_loss = loss_fn(y_train_pred, y_train_batch)
            epoch_train_loss += train_loss.item()
            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        mean_epoch_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(mean_epoch_train_loss)

        # Validation
        model.eval()
        epoch_valid_loss = 0.
        with torch.no_grad():
            for X_valid_batch, y_valid_batch in valid_loader:
                X_valid_batch, y_valid_batch = X_valid_batch.to(device), y_valid_batch.to(device)
                y_valid_pred = model(X_valid_batch)
                valid_loss = loss_fn(y_valid_pred, y_valid_batch)
                epoch_valid_loss += valid_loss.item()
        mean_epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        
        # Early stopping
        if epoch == 0:
            previous_loss = mean_epoch_valid_loss
            previous_model = deepcopy(model)
            valid_losses.append(mean_epoch_valid_loss)
        else: 
            if mean_epoch_valid_loss > previous_loss:
                model = deepcopy(previous_model)
                print(f"Training stopped after epoch {epoch + 1}, Training Loss: {mean_epoch_train_loss:.4f}, Valid Loss: {mean_epoch_valid_loss:.4f}.")
                break
            else: 
                previous_loss = mean_epoch_valid_loss
                previous_model = deepcopy(model)
                valid_losses.append(mean_epoch_valid_loss)
            
        print(f"Epoch {epoch + 1}/{n_epochs}, Training Loss: {mean_epoch_train_loss:.4f}, Valid Loss: {mean_epoch_valid_loss:.4f}")

    return (train_losses, valid_losses)

In [ ]:
model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse = nn.MSELoss()
n_epochs = 30

In [ ]:
train_losses_early_stop, valid_losses_early_stop = train_with_early_stopping(model, optimizer, mse, train_loader, valid_loader, n_epochs)

This basic version of early stopping might be to rigid and you might want to add a `patience` parameter to control the number of epochs without progress in validation loss seen before stopping the training. 

You can find a more advanced implementation of early stopping [here](https://www.geeksforgeeks.org/how-to-handle-overfitting-in-pytorch-models-using-early-stopping/).
The PyTorch Ignite and Lightning libraries also offer a more ready to use `EarlyStopping` class, but they are based on a different high-level API specific to these libraries. Do not hesitate to explore these, with ChatGPT as an assitant!

## Coding time 2

- Hyperparameter tuning with Optuna
- Learning rate schedule
- Batch norm

### 1. Hyperparameter tuning with Optuna

Several dedicated libraries can be used for hyperparamter tuning with PyTorch models. For example:
- [Optuna](https://optuna.org/)
- [Ray Tune](https://docs.ray.io/)
- [Hyperopt](https://hyperopt.github.io/hyperopt/)

We will see an example with using Optuna.
  

We first need to create a function that takes a `Trial` object and use it to have Optuna suggest hyperparameter values to try, train and evaluate a model with the selected hyperparameter values, and return the corresponding validation performance using a given evaluation metric. 

We start by trying to optimise the learning rate value (float) and the number of neurons in each hidden layer (int). We use the MSE loss as evaluation metric (for classification, we could use accuracy or another target metric).

Note: it is also possible to tune categorical hyperparameters with Optuna, using the `suggest_categorical()` method. 

In [ ]:
def objective(trial, train_loader, valid_loader):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True) # Suggest a float value in the chosen range
    n_hidden1 = trial.suggest_int("n_hidden1", 100, 300) # Suggest an int value in the chosen range
    n_hidden2 = trial.suggest_int("n_hidden2", 50, 200) # Suggest an int value in the chosen range
    n_hidden3 = trial.suggest_int("n_hidden3", 20, 100) # Suggest an int value in the chosen range
    model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=n_hidden1, n_hidden2=n_hidden2, n_hidden3=n_hidden3, n_outputs=1).to(device)
    mse = nn.MSELoss()
    n_epochs = 30
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    _ , valid_losses = train(model, optimizer, mse, train_loader, valid_loader, n_epochs)
    return valid_losses[-1]

Then, we need to create `Sampler` object specifying the algorithm we want to use for the optimisation. Some algorithms might select purely random values at each trial, some will use past information to guide the search. You can have a look at the different Optuna sampler algorithm options in the [documentation](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html).

Then, we need to create a `Study` object and pass the sampler as an argument, as well as the `direction` of the optimisation. In our case, we want to minimise the MSE used as evalaution metric. 

Finally, we can start the hyperparameter tuning by calling the `optimize` method and passing the objective function and number of trial as arguments. The number of trials correspond to the number of hyperparameter combinations the algorithm will try. The highest this number is, the more chance you have to find good hyperparameter values, but the longer it will take.

To pass the training and validations sets loader to the objective function, we can use the `functools.partial()` function which allows to provide default values for the objective fucntion arguments. 

In [ ]:
sampler = optuna.samplers.TPESampler() # Use the Tree-structured Parzen Estimator algorithm for the optimisation
study = optuna.create_study(direction="minimize", sampler=sampler)
objective_with_data = partial(objective, train_loader=train_loader, valid_loader=valid_loader)
study.optimize(objective_with_data, n_trials=5)

The best hyperparameter values are found in `study.best_params` and the corresponding best evaluation metric value in `study.best_value`.

In [ ]:
study.best_params

In [ ]:
study.best_value

**TODO**: Try to let it run for longer and see if it can find significantly better values!

To speed up the process, you can use a `Pruner` to stop trials that are leading nowehere (i.e., whose performance is below previously seen trials). For example, the `MedianPruner` will stop a trial if its performance is below the median of previous trials. The pruner needs to warmup first over a couple of trials set with the `n_startup_trials` parameter (5 by default). After that, it will monitor the performance every few epoch (set with the `interval_steps` parameter), after a few first warmup epoch (set with the `n_warmup_steps` parameter). For each of the controlled epochs, it verifies if the perfomance is better than the median perfomance at the same epoch for previous completed trials. 


In [ ]:
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0, interval_steps=1)
study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)

To use the pruner, you need to pass information about the validation performance after each epoch of training during a trial. In our case, we need to modify the train function.

In [ ]:
def train_for_pruner(model, optimizer, loss_fn, train_loader, valid_loader, n_epochs):
    train_losses = []
    valid_losses = []

    for epoch in range(n_epochs):
        #Training
        model.train()
        epoch_train_loss = 0.
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            y_train_pred = model(X_train_batch)
            train_loss = loss_fn(y_train_pred, y_train_batch)
            epoch_train_loss += train_loss.item()
            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        mean_epoch_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(mean_epoch_train_loss)

        # Validation
        model.eval()
        epoch_valid_loss = 0.
        with torch.no_grad():
            for X_valid_batch, y_valid_batch in valid_loader:
                X_valid_batch, y_valid_batch = X_valid_batch.to(device), y_valid_batch.to(device)
                y_valid_pred = model(X_valid_batch)
                valid_loss = loss_fn(y_valid_pred, y_valid_batch)
                epoch_valid_loss += valid_loss.item()
        mean_epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        valid_losses.append(mean_epoch_valid_loss)

        # Pass information to Optuna about the validation performance for the last epoch
        trial.report(valid_losses[-1], epoch)
        # Raise an exception to stop the training if Optuna's pruner decides to prune the current trial
        if trial.should_prune():
            raise optuna.TrialPruned()

        print(f"Epoch {epoch + 1}/{n_epochs}, Training Loss: {mean_epoch_train_loss:.4f}, Valid Loss: {mean_epoch_valid_loss:.4f}")

    return (train_losses, valid_losses)

In [ ]:
def objective(trial, train_loader, valid_loader):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True) # Suggest a float value in the chosen range
    n_hidden1 = trial.suggest_int("n_hidden1", 100, 300) # Suggest an int value in the chosen range
    n_hidden2 = trial.suggest_int("n_hidden2", 50, 200) # Suggest an int value in the chosen range
    n_hidden3 = trial.suggest_int("n_hidden3", 20, 100) # Suggest an int value in the chosen range
    model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=n_hidden1, n_hidden2=n_hidden2, n_hidden3=n_hidden3, n_outputs=1).to(device)
    mse = nn.MSELoss()
    n_epochs = 30
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    _ , valid_losses = train_for_pruner(model, optimizer, mse, train_loader, valid_loader, n_epochs) # NEW: Changed the train_for_pruner to use for pruning
    return valid_losses[-1]

In [ ]:
study.optimize(objective_with_data, n_trials=10) # Runs 5 trials without pruning and then activates the pruning until reaching 10 trials

You can read more about Optuna range of pruners in the [documentation](https://optuna.readthedocs.io/en/stable/reference/pruners.html).

### 2. Learning rate schedule

The `torch.optim` module contains a sub-module called `lr_scheduler` which allows you to use several types of schedulers.

First, we need to modify the training loop to accept a scheduler as parameter, and update the scheduler (i.e., the learning rate) during the training, in a similar fashion we update the optimiser.

In [ ]:
def train_with_scheduler(model, optimizer, scheduler, loss_fn, train_loader, valid_loader, n_epochs):
    train_losses = []
    valid_losses = []

    for epoch in range(n_epochs):
        #Training
        model.train()
        epoch_train_loss = 0.
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            y_train_pred = model(X_train_batch)
            train_loss = loss_fn(y_train_pred, y_train_batch)
            epoch_train_loss += train_loss.item()
            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        mean_epoch_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(mean_epoch_train_loss)

        # Scheduler update for every epoch (can be moved inside epoch loop for updates at every batch)
        before_lr = optimizer.param_groups[0]["lr"]
        scheduler.step() # Take a step of scheduler at the end of the epoch
        after_lr = optimizer.param_groups[0]["lr"]
        
        
        # Validation
        model.eval()
        epoch_valid_loss = 0.
        with torch.no_grad():
            for X_valid_batch, y_valid_batch in valid_loader:
                X_valid_batch, y_valid_batch = X_valid_batch.to(device), y_valid_batch.to(device)
                y_valid_pred = model(X_valid_batch)
                valid_loss = loss_fn(y_valid_pred, y_valid_batch)
                epoch_valid_loss += valid_loss.item()
        mean_epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        valid_losses.append(mean_epoch_valid_loss)
        
        print(f"Epoch {epoch + 1}/{n_epochs}, Training Loss: {mean_epoch_train_loss:.4f}, Valid Loss: {mean_epoch_valid_loss:.4f}")
        print(f"Epoch {epoch  + 1}: SGD lr {before_lr} -> {after_lr}")

    return (train_losses, valid_losses)

We then need to instantiate a scheduler before training and "link" it to the optimiser.

Below, we use a linear scheduler which will decay the learning rate linearly from the original leanring rate value to 50% of its original value over 20 epochs. After 20 epochs, the learning rate will remain constant at 50% of its original value. 

In [ ]:
model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
n_epochs = 30
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=20)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=2, T_mult=2, eta_min=0.001)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=n_epochs) # Need to move scheduler update in epoch loop
mse = nn.MSELoss()

In [ ]:
train_losses_lr_sched, valid_losses_lr_sched = train_with_scheduler(model, optimizer, scheduler, mse, train_loader, valid_loader, n_epochs)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_losses_relu, label = "ReLU no lr schedule Train Loss")
plt.plot(valid_losses_relu, label = "ReLU no lr schedule reg Valid Loss")
plt.legend()
plt.grid()
plt.subplot(2, 1, 2)
plt.plot(train_losses_lr_sched, label = "ReLU lr schedule Train Loss")
plt.plot(valid_losses_lr_sched, label = "ReLU lr schedule Valid Loss")
plt.legend()
plt.grid()
plt.show()

PyTorch provide a lot of different learning rate schedules you can use. Some examples are shown in [this Kaggle notebook](https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling). 

**TODO**: Try to use other learning rate schedules and see the impact on training. Some schedules alternate increasing and decreasing the learning rate to try to escape local optimum (e.g., `CosineAnnealingLR` or `OneCycleLR`).

### 3. Batch-normalisation

Let's include batch normalisation layers in our network. The authors of the method advocate to place them before the activation function of each layer, but this is a debated issues. In the case below, we placed them after the activation functions as it seems to work better. The choice seems to depend on your network architecture, therefore you can try both and see what works best fro you model.

In [ ]:
class MLP_BN(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_hidden3, n_outputs):
      super().__init__()
      self.mlp = nn.Sequential(
          nn.Flatten(),
          nn.Linear(n_inputs, n_hidden1),
          nn.ReLU(),
          nn.BatchNorm1d(n_hidden1),
          nn.Linear(n_hidden1, n_hidden2),
          nn.ReLU(),
          nn.BatchNorm1d(n_hidden2),
          nn.Linear(n_hidden2, n_hidden3),
          nn.ReLU(),
          nn.BatchNorm1d(n_hidden3),
          nn.Linear(n_hidden3, n_outputs)
          )
    def forward(self, X):
        return self.mlp(X)

*Remarks*: 
- As batch normalisation is behaving differently between training and evaluation, it is crutial to include the `model.train()` and `model.eval()` instructions in your training loop, to switch the model in training and evaluation mode, as we have already done.
- For models dealing with data in 2 or 3 dimensions (e.g., convolutional neural networks work with 2D images), you can use `nn.BatchNorm2d` or `nn.BatchNorm3d`.

In [ ]:
model = MLP_BN(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse = nn.MSELoss()
n_epochs = 30

In [ ]:
train_losses_bn, valid_losses_bn = train(model, optimizer, mse, train_loader, valid_loader, n_epochs)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_losses_relu, label = "ReLU no BN reg Train Loss")
plt.plot(valid_losses_relu, label = "ReLU no BN reg Valid Loss")
plt.legend()
plt.grid()
plt.subplot(2, 1, 2)
plt.plot(train_losses_bn, label = "ReLU BN reg Train Loss")
plt.plot(valid_losses_bn, label = "ReLU BN reg Valid Loss")
plt.legend()
plt.grid()
plt.show()

## Coding Time 3: Faster optimisers

The is a range of optimisers you can use with PyTorch, from the module `torch.optim`. We only used SGD and Adam (to train the image classifier in Lectutorial 2), but you can try a range of other optimisers such as RMSProp or AdaGrad. You can find the list of optimisers available in the `torch.optim` documentation. 

To change the optmiser, you simply need to pass a different one to your training function. Be mindful that different optimisers might have different hyper-parameters you can tune. For example, you can try to change the betas values for the ADAM optimiser:

In [ ]:
model = MLP_ReLU(n_inputs=X_train.shape[1], n_hidden1=200, n_hidden2=100, n_hidden3 = 50, n_outputs=1).to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999)) # Changed to ADAM optimiser
mse = nn.MSELoss()
n_epochs = 30

In [ ]:
train_losses_adam, valid_losses_adam = train(model, optimizer, mse, train_loader, valid_loader, n_epochs)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(train_losses_relu, label = "ReLU SGD Train Loss")
plt.plot(valid_losses_relu, label = "ReLU SGD Valid Loss")
plt.legend()
plt.grid()
plt.subplot(2, 1, 2)
plt.plot(train_losses_adam, label = "ReLU Adam Train Loss")
plt.plot(valid_losses_adam, label = "ReLU Adam Valid Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(train_losses_relu, label = "ReLU SGD Train Loss")
plt.plot(valid_losses_relu, label = "ReLU SGD Valid Loss")
plt.plot(train_losses_adam, label = "ReLU Adam Train Loss")
plt.plot(valid_losses_adam, label = "ReLU Adam Valid Loss")
plt.legend()
plt.grid()
plt.show()

**TODO**: Try other optimisers and see how they behave with this model and problem.